In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display
from ipyleaflet import (
    Map, 
    Marker, 
    basemaps, 
    basemap_to_tiles
)

In [2]:
# tx=-4.77, ty=20.48   -> (55.929142, 37.517628)
# tx=-97.27, ty=107.93 -> (55.927658, 37.518561)
# tx=-87.20, ty=48.72  -> (55.928269, 37.518217)
# tx=88.88, ty=-10.31  -> (55.929603, 37.516117)
# tx=140.10, ty=-38.89 -> (55.930106, 37.515986)
# tx=44.77, ty=20.88   -> (55.929475, 37.516956)
# Let:
# lat = a1 * tx + b1 * ty + c1
# lon = a2 * tx + b2 * ty + c2

local = np.array([
    [-4.77,  20.48],
    [-97.27, 107.93],
    [-87.20,  48.72],
    [ 88.88, -10.31],
    [140.10, -38.89],
    [ 44.77,  20.88]
])

geo = np.array([
    [55.929142, 37.517628],
    [55.927658, 37.518561],
    [55.928269, 37.518217],
    [55.929603, 37.516117],
    [55.930106, 37.515986],
    [55.929475, 37.516956]
])

A = np.hstack([local, np.ones((local.shape[0], 1))])
lat_coeffs, _, _, _ = np.linalg.lstsq(A, geo[:,0], rcond=None)
lon_coeffs, _, _, _ = np.linalg.lstsq(A, geo[:,1], rcond=None)

print(f"lat = {lat_coeffs[0]:.8f} * tx + {lat_coeffs[1]:.8f} * ty + {lat_coeffs[2]:.8f}")
print(f"lon = {lon_coeffs[0]:.8f} * tx + {lon_coeffs[1]:.8f} * ty + {lon_coeffs[2]:.8f}")

lat = 0.00000473 * tx + -0.00000916 * ty + 55.92920259
lon = -0.00001008 * tx + 0.00000206 * ty + 37.51733509


In [3]:
IMG_DIR = Path("../data/test/07_2023-10-04-day/front_cam")
TRACK_CSV = Path("../data/test/07_2023-10-04-day/track.csv")
df = pd.read_csv(
    TRACK_CSV,
    dtype={'front_cam_ts': str}
)
img_files = [IMG_DIR / f"{row['front_cam_ts']}.jpg" for _, row in df.iterrows()]
coords = df[['tx', 'ty']].values

In [4]:
def fit_affine(local_coords, geo_coords):
    A = np.hstack([local_coords, np.ones((local_coords.shape[0], 1))])
    lat_coeffs, _, _, _ = np.linalg.lstsq(A, geo_coords[:, 0], rcond=None)
    lon_coeffs, _, _, _ = np.linalg.lstsq(A, geo_coords[:, 1], rcond=None)
    return lat_coeffs, lon_coeffs

def local_to_geo(tx, ty, lat_coeffs, lon_coeffs):
    lat = lat_coeffs[0] * tx + lat_coeffs[1] * ty + lat_coeffs[2]
    lon = lon_coeffs[0] * tx + lon_coeffs[1] * ty + lon_coeffs[2]
    return lat, lon

def geo_to_local(lat, lon, lat_coeffs, lon_coeffs):
    A = np.array([[lat_coeffs[0], lat_coeffs[1]], [lon_coeffs[0], lon_coeffs[1]]])
    b = np.array([lat - lat_coeffs[2], lon - lon_coeffs[2]])
    tx, ty = np.linalg.solve(A, b)
    return tx, ty

In [5]:
img_name_label = widgets.Label()
coords_label = widgets.Label()

def show_image(img_path):
    img = cv2.imread(str(img_path))
    if img is None:
        img = np.zeros((360, 360, 3), dtype=np.uint8)
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (360, 360))
    return widgets.Image(value=cv2.imencode('.png', img)[1].tobytes(), format='png', width=360, height=360)

idx = 0
img_widget = show_image(img_files[idx])
center = (coords[idx,1], coords[idx,0])
m = Map(center=center, zoom=18, basemap=basemaps.OpenStreetMap.Mapnik, scroll_wheel_zoom=True)
marker = Marker(location=center, draggable=True)
m.add_layer(marker)

def update_widgets():
    global idx
    img_widget.value = cv2.imencode('.png', cv2.cvtColor(cv2.imread(str(img_files[idx])), cv2.COLOR_BGR2RGB))[1].tobytes()
    tx, ty = coords[idx,0], coords[idx,1]
    lat, lon = local_to_geo(tx, ty, lat_coeffs, lon_coeffs)
    marker.location = (lat, lon)
    m.center = marker.location
    img_name_label.value = f"Файл: {img_files[idx].name}"
    coords_label.value = f"Локальные: tx={tx:.2f}, ty={ty:.2f} | Гео: lat={lat:.6f}, lon={lon:.6f}"

def on_next(b):
    global idx
    if idx < len(img_files) - 1:
        idx += 1
        update_widgets()

def on_prev(b):
    global idx
    if idx > 0:
        idx -= 1
        update_widgets()

def on_marker_drag(event=None, **kwargs):
    lat, lon = marker.location
    tx, ty = geo_to_local(lat, lon, lat_coeffs, lon_coeffs)
    coords[idx,0] = tx
    coords[idx,1] = ty
    df.loc[idx, 'tx'] = tx
    df.loc[idx, 'ty'] = ty
    coords_label.value = f"Локальные: tx={tx:.2f}, ty={ty:.2f} | Гео: lat={lat:.6f}, lon={lon:.6f}"

marker.observe(on_marker_drag, names='location')

btn_prev = widgets.Button(description="← Предыдущее")
btn_next = widgets.Button(description="Следующее →")
btn_prev.on_click(on_prev)
btn_next.on_click(on_next)

info_box = widgets.VBox([img_name_label, coords_label])
ui = widgets.HBox([widgets.VBox([img_widget, info_box]), m])
controls = widgets.HBox([btn_prev, btn_next])
display(ui, controls)

def save_annotations():
    df[['tx', 'ty']].to_csv("annotated_coords.csv", index=False)

update_widgets()